In [1]:
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine, insert

In [2]:
df = pd.read_csv('wine_scraped_dataset.csv', index_col=0)

## Assess 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23822 entries, 0 to 23821
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  23822 non-null  object 
 1   price         23822 non-null  object 
 2   varietal      23822 non-null  object 
 3   origin        23819 non-null  object 
 4   rating        23822 non-null  float64
 5   rating_count  23822 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.3+ MB


In [4]:
df.head()

,product_name,price,varietal,origin,rating,rating_count
0,Dom Perignon Vintage with Gift Box 2010,199,Vintage Sparkling Wine,"Champagne, France",4.5,42
1,Veuve Clicquot Yellow Label Brut,59,Non-Vintage Sparkling Wine,"Champagne, France",4.4,1138
2,Duckhorn Napa Valley Cabernet Sauvignon 2017,78,Cabernet Sauvignon,"Napa Valley, California",4.3,62
3,Caymus Special Selection Cabernet Sauvignon 2016,180,Cabernet Sauvignon,"Napa Valley, California",4.6,108
4,Quintessa 2016,199,Cabernet Sauvignon,"Rutherford, Napa Valley, California",4.7,73


In [5]:
df['varietal'].sort_values().unique()

array(['Agiorgitiko', 'Aglianico', 'Albarino', 'Alicante Bouschet',
       'Arneis', 'Assyrtiko', 'Baga', 'Barbera', 'Blaufrankisch', 'Bobal',
       'Bordeaux Red Blends', 'Bordeaux White Blends', 'Cabernet Franc',
       'Cabernet Sauvignon', 'Carignan', 'Carmenere', 'Chardonnay',
       'Chenin Blanc', 'Cinsault', 'Corvina', 'Dolcetto', 'Fiano',
       'Friulano', 'Fruit Wine', 'Furmint', 'Gamay', 'Garganega',
       'Gewurztraminer', 'Godello', 'Greco', 'Grenache', 'Grenache Blanc',
       'Gruner Veltliner', 'Lagrein', 'Madeira', 'Malbec', 'Malvasia',
       'Marsanne', 'Melon de Bourgogne', 'Mencia', 'Merlot',
       'Montepulciano', 'Mourvedre', 'Muscat', 'Nebbiolo',
       'Nerello Mascalese', "Nero d'Avola", 'Non-Vintage Sparkling Wine',
       'Other Dessert', 'Other Red Blends', 'Other Red Wine',
       'Other White Blends', 'Other White Wine', 'Petit Verdot',
       'Petite Sirah', 'Pinot Blanc', 'Pinot Gris/Grigio', 'Pinot Noir',
       'Pinotage', 'Port', 'Red Sparkling W

### Cleaning 

The following items have been indentified as tidiness/ quality issues:
- 'origin' is missing three values
- separate 'origin' into 'region' and 'appelation'
- Remove non wines from varietals: 'Port' 'Non-Vintage Sparkling Wine' 'Vintage Sparkling Wine'
- Seperate 'year' out of 'product_name' and drop the ones that don't have a year?
- turn remove commas and turn price into int

### 'origin' is missing three values 

In [6]:
df = df[df['origin'].notna()]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23819 entries, 0 to 23821
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  23819 non-null  object 
 1   price         23819 non-null  object 
 2   varietal      23819 non-null  object 
 3   origin        23819 non-null  object 
 4   rating        23819 non-null  float64
 5   rating_count  23819 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 1.3+ MB


###  separate 'origin' into 'region' and 'appelation'

In [8]:
df_or = df.copy()

In [9]:
df_or = df_or.origin.str.rsplit(',').str[-2:]

In [10]:
df_or = df_or.to_frame()

In [11]:
df_or = pd.DataFrame(df.origin.str.rsplit(',', 1).tolist(),
                                 columns = ['appellation','region'])

In [12]:
df_or2 = df_or.appellation.str.rsplit(',').str[-1:]

In [13]:
df_or2 = df_or2.to_frame()

In [14]:
df_or2['appellation'] = df_or2['appellation'].astype(str)

In [15]:
type(df_or2['appellation'])

pandas.core.series.Series

In [16]:
df_merged = df_or.assign(appellation=df_or2['appellation'])

In [17]:
df.drop('origin', axis = 1, inplace = True)

In [18]:
df = pd.concat([df, df_merged], axis=1)

In [19]:
df.head(2)

,product_name,price,varietal,rating,rating_count,appellation,region
0,Dom Perignon Vintage with Gift Box 2010,199,Vintage Sparkling Wine,4.5,42.0,['Champagne'],France
1,Veuve Clicquot Yellow Label Brut,59,Non-Vintage Sparkling Wine,4.4,1138.0,['Champagne'],France


In [20]:
df.dropna(inplace = True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22685 entries, 0 to 23818
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  22685 non-null  object 
 1   price         22685 non-null  object 
 2   varietal      22685 non-null  object 
 3   rating        22685 non-null  float64
 4   rating_count  22685 non-null  float64
 5   appellation   22685 non-null  object 
 6   region        22685 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.4+ MB


In [22]:
df['appellation'].sort_values().unique()

array(['["Valle d\'Aosta"]', "[' Bordeaux']", "[' Burgundy']",
       "[' Central Coast']", "[' Columbia Valley']",
       "[' Friuli-Venezia Giulia']", "[' Loire']", "[' Mendocino']",
       "[' Mendoza']", "[' Napa Valley']", "[' New South Wales']",
       "[' New York']", "[' North Coast']", "[' Piedmont']",
       "[' Prosecco']", "[' Rapel Valley']", "[' Rhone']", "[' Salta']",
       "[' San Antonio Valley (Chile)']", "[' Sierra Foothills']",
       "[' Sonoma County']", "[' South Australia']",
       "[' South of France']", "[' Southwest']",
       "[' Trentino-Alto Adige']", "[' Tuscany']", "[' Veneto']",
       "[' Victoria']", "[' Western Australia']",
       "[' Willamette Valley']", "['Abruzzo']", "['Aconcagua Valley']",
       "['Alentejo']", "['Alsace']", "['Auckland']", "['Basilicata']",
       "['Bierzo']", "['Bordeaux']", "['Burgenland']", "['Burgundy']",
       "['Campania']", "['Canterbury']", "['Carneros']",
       "['Casablanca Valley']", "['Central Coast']", "['Ce

In [23]:
df['region'].sort_values().unique()

array([' Argentina', ' Australia', ' Austria', ' California', ' Chile',
       ' France', ' Germany', ' Greece', ' Italy', ' New Zealand',
       ' Oregon', ' Other U.S.', ' Portugal', ' South Africa', ' Spain',
       ' Washington'], dtype=object)

### seperate 'year' out of 'product_name' and drop the ones that don't have a year? 

In [24]:
# this removes the parenthesis and what's between them. made it easier to extract year. 
df['product_name'] = df['product_name'].str.replace(r"\(.*?\)", "")

In [25]:
# the following only extract the digit from left to right. 
df['year']= df.product_name.str.extract('(\d+)')

In [26]:
df['product_name'] = df['product_name'].str[:-5] #.str[-1:]

In [27]:
df.dropna(inplace = True)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22510 entries, 0 to 23818
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  22510 non-null  object 
 1   price         22510 non-null  object 
 2   varietal      22510 non-null  object 
 3   rating        22510 non-null  float64
 4   rating_count  22510 non-null  float64
 5   appellation   22510 non-null  object 
 6   region        22510 non-null  object 
 7   year          22510 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.5+ MB


In [29]:
# turn the string year into an integer
df['year'] = df['year'].astype(int)

In [30]:
df['year'].sort_values().unique()

array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
         11,   12,   13,   14,   15,   16,   17,   20,   21,   22,   23,
         24,   25,   27,   28,   29,   30,   32,   36,   40,   41,   43,
         45,   50,   51,   55,   60,   66,   75,   81,   84,   88,   94,
        100,  101,  115,  150,  170,  203,  239,  302,  337,  375,  389,
        407,  459,  620,  707,  890,  902,  904, 1003, 1102, 1147, 1206,
       1302, 1403, 1406, 1522, 1614, 1752, 1792, 1830, 1843, 1850, 1855,
       1860, 1863, 1875, 1898, 1902, 1906, 1908, 1920, 1927, 1937, 1951,
       1955, 1959, 1961, 1963, 1964, 1965, 1966, 1967, 1968, 1970, 1971,
       1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985,
       1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996,
       1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007,
       2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018,
       2019, 2020, 2134, 4002, 5500, 7200])

In [31]:
# search for the weird year entries above
df.loc[df['year'] == 1961]

,product_name,price,varietal,rating,rating_count,appellation,region,year
11503,Pewsey Vale 1961 Block Riesling,49,Riesling,0.0,0.0,[' Bordeaux'],France,1961


In [32]:
# drop anything that is older than 
df = df[df['year'] > 1937]  

In [33]:
df.shape

(22004, 8)

### remove commas and turn price into int 

In [34]:
df['price'] = df['price'].str.replace(',', '')

In [35]:
df['price'].min(), df['price'].max()

('100', '9999')

In [36]:
df['price'] = df['price'].astype(int)

In [37]:
df.dtypes

product_name     object
price             int64
varietal         object
rating          float64
rating_count    float64
appellation      object
region           object
year              int64
dtype: object

### look for zero and non values in the ratings

In [38]:
df['rating'].sort_values().unique()

array([0. , 1.6, 1.9, 2.2, 2.4, 2.6, 2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3,
       3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6,
       4.7, 4.8, 4.9, 5. ])

In [39]:
df.groupby('rating').size()

rating
0.0    17893
1.6        1
1.9        1
2.2        1
2.4        1
2.6        3
2.7        1
2.8        2
2.9        4
3.0       10
3.1       14
3.2       19
3.3       28
3.4       41
3.5       44
3.6       97
3.7      115
3.8      182
3.9      239
4.0      290
4.1      343
4.2      442
4.3      382
4.4      383
4.5      354
4.6      322
4.7      243
4.8      194
4.9      165
5.0      190
dtype: int64

In [40]:
df['rating_count'].sort_values().unique()

array([  0.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,
        15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,
        37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
        48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,
        59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,
        70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,
        81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.,  90.,  92.,
        93.,  94.,  95.,  97.,  99., 100., 101., 102., 103., 105., 106.,
       107., 108., 109., 110., 112., 113., 114., 115., 116., 118., 119.,
       122., 123., 125., 127., 128., 129., 130., 131., 132., 136., 137.,
       138., 139., 140., 143., 144., 146., 147., 148., 149., 150., 151.,
       152., 156., 157., 158., 160., 164., 166., 168., 169., 172., 174.,
       176., 177., 180., 183., 184., 185., 188., 19

In [41]:
df.groupby('rating_count').size()

rating_count
0.0      17893
5.0        556
6.0        444
7.0        357
8.0        269
         ...  
556.0        1
614.0        1
663.0        3
672.0        3
750.0        1
Length: 198, dtype: int64

## Dataframes

### Master  

In [42]:
df.shape

(22004, 8)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22004 entries, 0 to 23818
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  22004 non-null  object 
 1   price         22004 non-null  int64  
 2   varietal      22004 non-null  object 
 3   rating        22004 non-null  float64
 4   rating_count  22004 non-null  float64
 5   appellation   22004 non-null  object 
 6   region        22004 non-null  object 
 7   year          22004 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 1.5+ MB


In [44]:
df.head()

,product_name,price,varietal,rating,rating_count,appellation,region,year
0,Dom Perignon Vintage with Gift Box,199,Vintage Sparkling Wine,4.5,42.0,['Champagne'],France,2010
2,Duckhorn Napa Valley Cabernet Sauvignon,78,Cabernet Sauvignon,4.3,62.0,['Napa Valley'],California,2017
3,Caymus Special Selection Cabernet Sauvignon,180,Cabernet Sauvignon,4.6,108.0,['Napa Valley'],California,2016
4,Quintessa,199,Cabernet Sauvignon,4.7,73.0,[' Napa Valley'],California,2016
5,Joseph Phelps Insignia,300,Bordeaux Red Blends,4.6,81.0,['Napa Valley'],California,2016


In [45]:
df.to_csv('wine_master_dataset.csv')

###  Wines with Ratings 

In [46]:
df_ratings_only = df.copy()

In [47]:
df_ratings_only = df_ratings_only[df_ratings_only['rating_count'] > 0]  

In [48]:
df_ratings_only.shape

(4111, 8)

In [49]:
df_ratings_only.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4111 entries, 0 to 23786
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  4111 non-null   object 
 1   price         4111 non-null   int64  
 2   varietal      4111 non-null   object 
 3   rating        4111 non-null   float64
 4   rating_count  4111 non-null   float64
 5   appellation   4111 non-null   object 
 6   region        4111 non-null   object 
 7   year          4111 non-null   int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 289.1+ KB


In [50]:
df_ratings_only.head()

,product_name,price,varietal,rating,rating_count,appellation,region,year
0,Dom Perignon Vintage with Gift Box,199,Vintage Sparkling Wine,4.5,42.0,['Champagne'],France,2010
2,Duckhorn Napa Valley Cabernet Sauvignon,78,Cabernet Sauvignon,4.3,62.0,['Napa Valley'],California,2017
3,Caymus Special Selection Cabernet Sauvignon,180,Cabernet Sauvignon,4.6,108.0,['Napa Valley'],California,2016
4,Quintessa,199,Cabernet Sauvignon,4.7,73.0,[' Napa Valley'],California,2016
5,Joseph Phelps Insignia,300,Bordeaux Red Blends,4.6,81.0,['Napa Valley'],California,2016


### Wines with Bonafide Varietals

In [51]:
df_varietals = df.copy()

In [52]:
non_varietals = ['Port', 'Non-Vintage Sparkling Wine', 'Vintage Sparkling Wine']
# Delete these row indexes from dataFrame
df_varietals = df_varietals[~df_varietals['varietal'].isin(non_varietals)]

In [53]:
# this is the df with the non-wine varietals
df_varietals['varietal'].sort_values().unique()

array(['Agiorgitiko', 'Aglianico', 'Albarino', 'Alicante Bouschet',
       'Arneis', 'Assyrtiko', 'Baga', 'Barbera', 'Blaufrankisch', 'Bobal',
       'Bordeaux Red Blends', 'Bordeaux White Blends', 'Cabernet Franc',
       'Cabernet Sauvignon', 'Carignan', 'Carmenere', 'Chardonnay',
       'Chenin Blanc', 'Corvina', 'Dolcetto', 'Fiano', 'Friulano',
       'Fruit Wine', 'Furmint', 'Gamay', 'Garganega', 'Gewurztraminer',
       'Godello', 'Greco', 'Grenache', 'Grenache Blanc',
       'Gruner Veltliner', 'Lagrein', 'Madeira', 'Malbec', 'Marsanne',
       'Melon de Bourgogne', 'Mencia', 'Merlot', 'Montepulciano',
       'Mourvedre', 'Muscat', 'Nebbiolo', 'Nerello Mascalese',
       "Nero d'Avola", 'Other Dessert', 'Other Red Blends',
       'Other Red Wine', 'Other White Blends', 'Other White Wine',
       'Petit Verdot', 'Petite Sirah', 'Pinot Blanc', 'Pinot Gris/Grigio',
       'Pinot Noir', 'Pinotage', 'Red Sparkling Wine', 'Rhone Red Blends',
       'Rhone White Blends', 'Riesling', 'R

In [54]:
df_varietals.shape

(21379, 8)

In [55]:
df_varietals.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21379 entries, 2 to 23818
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  21379 non-null  object 
 1   price         21379 non-null  int64  
 2   varietal      21379 non-null  object 
 3   rating        21379 non-null  float64
 4   rating_count  21379 non-null  float64
 5   appellation   21379 non-null  object 
 6   region        21379 non-null  object 
 7   year          21379 non-null  int64  
dtypes: float64(2), int64(2), object(4)
memory usage: 1.5+ MB


In [56]:
df_varietals.head()

,product_name,price,varietal,rating,rating_count,appellation,region,year
2,Duckhorn Napa Valley Cabernet Sauvignon,78,Cabernet Sauvignon,4.3,62.0,['Napa Valley'],California,2017
3,Caymus Special Selection Cabernet Sauvignon,180,Cabernet Sauvignon,4.6,108.0,['Napa Valley'],California,2016
4,Quintessa,199,Cabernet Sauvignon,4.7,73.0,[' Napa Valley'],California,2016
5,Joseph Phelps Insignia,300,Bordeaux Red Blends,4.6,81.0,['Napa Valley'],California,2016
6,Opus One,364,Bordeaux Red Blends,4.2,8.0,[' Napa Valley'],California,2017


# Store in SQL Database 

In [57]:
# Create SQLAlchemy Engine and empty bestofrt database
# bestofrt.db will not show up in the Jupyter Notebook dashboard yet
engine = create_engine('sqlite:///winewebscraping.db')

In [58]:
# Store cleaned master DataFrame ('df') in a table called master in marswind.db
# marswind.db will be visible now in the Jupyter Notebook dashboard
df.to_sql('master', engine, index=False)
df.to_sql('ratings', engine, index=False)
df.to_sql('varietals', engine, index=False)

In [59]:
df_db = pd.read_sql('SELECT * FROM varietals', engine)

In [60]:
df_db.head(3)

,product_name,price,varietal,rating,rating_count,appellation,region,year
0,Dom Perignon Vintage with Gift Box,199,Vintage Sparkling Wine,4.5,42.0,['Champagne'],France,2010
1,Duckhorn Napa Valley Cabernet Sauvignon,78,Cabernet Sauvignon,4.3,62.0,['Napa Valley'],California,2017
2,Caymus Special Selection Cabernet Sauvignon,180,Cabernet Sauvignon,4.6,108.0,['Napa Valley'],California,2016
